In [15]:
!pip install tqdm_notebook > /dev/null 2>&1

In [16]:
#Import Libraries

import csv
import random
import numpy as np
import pandas as pd

# Plotting
import matplotlib.pyplot as plt

# Machine Learning
from sklearn.preprocessing import normalize
import scipy.cluster.hierarchy as sch


%matplotlib inline

In [17]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  from tqdm.notebook import tqdm_notebook as tqdm
  tqdm().pandas()
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [18]:
base_path = "/content/drive/MyDrive/Colab Notebooks/data/"

# Data cleaning

In [19]:
df = pd.read_csv(base_path+"metadata.csv", low_memory=False)
df.info(verbose=True)
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497906 entries, 0 to 497905
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   cord_uid          497906 non-null  object 
 1   sha               172105 non-null  object 
 2   source_x          497906 non-null  object 
 3   title             497674 non-null  object 
 4   doi               281753 non-null  object 
 5   pmcid             182522 non-null  object 
 6   pubmed_id         241936 non-null  object 
 7   license           497906 non-null  object 
 8   abstract          363064 non-null  object 
 9   publish_time      497687 non-null  object 
 10  authors           484020 non-null  object 
 11  journal           464755 non-null  object 
 12  mag_id            0 non-null       float64
 13  who_covidence_id  197568 non-null  object 
 14  arxiv_id          6687 non-null    object 
 15  pdf_json_files    172105 non-null  object 
 16  pmc_json_files    13

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [23]:
# load the meta data from the CSV file and extract out fields that will be useful to our application

df = df[['title','abstract','authors','publish_time']]
df = df.drop_duplicates(subset='abstract', keep="first")
df=df.dropna()
df["abstract"] = df["abstract"].str.lower()
df['publish_time'] = pd.to_datetime(df['publish_time'])

KeyError: ignored

In [21]:
df.shape

(129900, 3)

In [22]:
df.head()

,title,abstract,sha
0,Clinical features of culture-proven Mycoplasma...,objective: this retrospective chart review des...,d1aafb70c066a2068b02786f8929fd9c900897fb
1,Nitric oxide: a pro-inflammatory mediator in l...,inflammatory diseases of the respiratory tract...,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d
2,Surfactant protein-D and pulmonary host defense,surfactant protein-d (sp-d) participates in th...,06ced00a5fc04215949aa72528f2eeaae1d58927
3,Role of endothelin-1 in lung disease,endothelin-1 (et-1) is a 21 amino acid peptide...,348055649b6b8cf2b9a376498df9bf41f7123605
4,Gene expression in epithelial cells in respons...,respiratory syncytial virus (rsv) and pneumoni...,5f48792a5fa08bed9f56016f4981ae2ca6031b32


In [9]:
from bs4 import BeautifulSoup
import html
import re
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', \
                "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', \
                'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", \
                'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', \
                'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', \
                'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', \
                'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
                'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', \
                'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', \
                'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', \
                'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', \
                'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", \
                'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", \
                'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', \
                "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', \
                "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", \
                'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"])

# ref: https://gist.github.com/sebleier/554280
# above stop words were used to remove any words that are not useful/influential for sentiment analysis/prediction. The above list/set is not conclusive by any means but it contains enough to simplify our model inputs

# data cleaning function that ensures that unnecssary elements are exluded from the journey of predicting sentiments. In the process, hashtags and mentions are removed, which could have been useful in a simpler stratefy implemented in exploratory analysis 
def clean_text(text):
  soup = BeautifulSoup(text, 'lxml') 
  text = soup.get_text() # gets rid of html tags  
  t = re.sub('^', '', text) #gets rid of the attribute '^' by replacing with space
  t = re.sub('[+]', '', t) # gets rid of the attribute '+' 
  t = re.sub('[/]', '', t) # gets rid of the attrubute '/'
  t = re.sub('[<>]', '', t) # gets rid of the attributes '<>'
  t = re.sub('[^A-Za-z0-9]+', ' ', t) # gets rid of any special charachters like #, @, -, etc. This is to make sure data is in its basic elements to ease the classification of tweet's political party in the next step 
  t = html.unescape(t) # replaces HTML charachter codes with ASCII equivalent
  t = re.sub(r"http\S+", "", t) # removes URL links
  t = t.lower() # normalizes the tweets to be all as lower case
  t = ' '.join(e.lower() for e in t.split() if e.lower() not in stopwords) # removes stopwords that are in the set defined above and replaces it with space
  return t

In [10]:
def clean_df(data):
  data['cleaned_title'] = data['title'].progress_apply(clean_text)
  data['cleaned_abstract'] = data['abstract'].progress_apply(clean_text)
  data['cleaned_authors'] = data['authors'].progress_apply(clean_text)
# clean_df(df)

In [11]:
def categorize_time(data):
  data['publish_year'] = data['publish_time'].dt.year
  data['publish_month'] = data['publish_time'].dt.month
  data['publish_date'] = data['publish_time'].dt.day
# categorize_time(df)

In [12]:
# df.to_csv(base_path + "preproced_metadata.csv")

# Data visualization

In [13]:
proced_df = pd.read_csv(base_path + "preproced_metadata.csv", low_memory=False)

In [14]:
proced_df.head()

,Unnamed: 0,title,abstract,authors,publish_time,cleaned_title,cleaned_abstract,cleaned_authors,publish_year,publish_month,publish_date
0,0,Clinical features of culture-proven Mycoplasma...,objective: this retrospective chart review des...,"Madani, Tariq A; Al-Ghamdi, Aisha A",2001-07-04,clinical features culture proven mycoplasma pn...,objective retrospective chart review describes...,madani tariq al ghamdi aisha,2001,7,4
1,1,Nitric oxide: a pro-inflammatory mediator in l...,inflammatory diseases of the respiratory tract...,"Vliet, Albert van der; Eiserich, Jason P; Cros...",2000-08-15,nitric oxide pro inflammatory mediator lung di...,inflammatory diseases respiratory tract common...,vliet albert van der eiserich jason p cross ca...,2000,8,15
2,2,Surfactant protein-D and pulmonary host defense,surfactant protein-d (sp-d) participates in th...,"Crouch, Erika C",2000-08-25,surfactant protein pulmonary host defense,surfactant protein sp participates innate resp...,crouch erika c,2000,8,25
3,3,Role of endothelin-1 in lung disease,endothelin-1 (et-1) is a 21 amino acid peptide...,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",2001-02-22,role endothelin 1 lung disease,endothelin 1 et 1 21 amino acid peptide divers...,fagan karen mcmurtry ivan f rodman david,2001,2,22
4,4,Gene expression in epithelial cells in respons...,respiratory syncytial virus (rsv) and pneumoni...,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",2001-05-11,gene expression epithelial cells response pneu...,respiratory syncytial virus rsv pneumonia viru...,domachowske joseph b bonville cynthia rosenber...,2001,5,11


## publish year distribution

In [ ]:
proced_df['publish_year'].plot.hist()
plt.xlabel('Publish year')  
plt.ylabel('Counts')
plt.title('Publish year distribution')
plt.show()

## Plot word frequency for abstract

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
def get_word_freq(data, top_N=20):
  preproced_data = data.str.cat(sep=' ')
  words = nltk.tokenize.word_tokenize(preproced_data)
  words_selected = [i for i in words if len(i) > 2]
  words_dist = nltk.FreqDist(words_selected)
  words_freq = pd.DataFrame(words_dist.most_common(top_N),
                      columns=['Word', 'Frequency'])
  return words_freq

In [ ]:
abstract_words_freq = get_word_freq(proced_df['cleaned_abstract'], top_N=20)

In [ ]:
abstract_words_freq

In [ ]:
abstract_words_freq.plot(x='Word', y='Frequency', kind='bar')

## Plot word frequency for title

In [ ]:
title_words_freq = get_word_freq(proced_df['cleaned_title'], top_N=20)

In [ ]:
title_words_freq

In [ ]:
title_words_freq.plot(x='Word', y='Frequency', kind='bar')

## Plot word frequency for author

In [ ]:
author_words_freq = get_word_freq(proced_df['cleaned_authors'], top_N=20)

In [ ]:
author_words_freq

In [ ]:
author_words_freq.plot(x='Word', y='Frequency', kind='bar')

# Model selection and fitting to the data